[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fireworks-ai/cookbook/blob/main/integrations/MCPHandle/Use_MCPHandle_with_Fireworks.ipynb)


# Fireworks AI + MCP Handle Integration

# <img src="../../static/fireworks-mcp_handle.png" width="500">


In this tutorial, we'll explore how to build an AI agent that integrates Fireworks AI's LLM with MCPHandle MCP Servers:


## Prerequisites

Before we begin, you'll need:

- **Fireworks AI API key** - see here [fireworks.ai](https://fireworks.ai/)
- **MCPHandle API key** - see here [mcp-handle.dev](https://www.mcp-handle.dev/)

Make sure to keep these API keys secure and never commit them to version control!


In [31]:
# Install the required packages
%pip install -qU fireworks-ai mcp_handle


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import json
from fireworks.client import Fireworks
from mcp_handle import MCPHandle
from mcp_handle.types import McpServerName, ToolFormat

# Set environment variables
os.environ["FIREWORKS_API_KEY"] = "fw_XXXXXXXXXXXXXX"  # Replace with your actual Fireworks API key
os.environ["MCP_HANDLE_API_KEY"] = "XXXXXXXXXXXXX"    # Replace with your actual MCPHandle API key


## Create AI Agent with MCP Integration

Now we'll create an intelligent agent that can use MCP servers through MCPHandle API. This agent will:

1. **Discover Tools**: Automatically find available tools from MCP servers
2. **Function Calling**: Use Fireworks AI's function calling capabilities
3. **Tool Execution**: Execute tools through MCPHandle API
4. **Smart Responses**: Generate intelligent responses based on tool results


In [34]:
class Agent:
    def __init__(self, fireworks_client, mcp_handle_client, mcp_server_url):
        self.fireworks = fireworks_client
        self.mcp_handle = mcp_handle_client
        self.mcp_server_url = mcp_server_url
        self.model = "accounts/fireworks/models/qwen2p5-72b-instruct"
        print(f"🤖 Agent initialized with model: {self.model}")
    
    def process_request(self, user_message):
        # 1. Get available tools
        mcp_tools = self.mcp_handle.mcp_server.list_tools(
            server_url=self.mcp_server_url,
            format=ToolFormat.OPENAI,
        )
        
        # 2. Call LLM with tools
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_message}
        ]
        
        response = self.fireworks.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=mcp_tools.tools
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # 3. If LLM wants to use tools
        if assistant_message.tool_calls:
            
            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)
                
                print(f"🛠️ Calling tool: {tool_name} with args: {tool_args}")
                # Call tool via MCPHandle SDK
                tool_result = self.mcp_handle.mcp_server.call_tools(
                    server_url=self.mcp_server_url,
                    tool_name=tool_name,
                    tool_args=tool_args,
                )
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(tool_result)
                })
            
            # 4. Get final response from LLM
            final_response = self.fireworks.chat.completions.create(
                model=self.model,
                messages=messages
            )
            return final_response.choices[0].message.content
        
        # If no tools needed, return the assistant message directly
        return assistant_message.content


## Use Case 1: Summarize YouTube Video

In [35]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=kPXvf2-C_Hs" # pick a video you like!

# 1. Initialize Fireworks AI client and MCPHandle client
fireworks_client = Fireworks(api_key=os.getenv("FIREWORKS_API_KEY"))
mcp_handle_client = MCPHandle(api_key=os.getenv("MCP_HANDLE_API_KEY"))

# 2. Create YouTube MCP server instance
youtube_mcp_instance = mcp_handle_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
)

# 3. Create an agent with YouTube MCP server
agent = Agent(fireworks_client, mcp_handle_client, youtube_mcp_instance.server_url)

response = agent.process_request(
    f"Summarize this YouTube video with timestamps: {YOUTUBE_VIDEO_URL}"
)

print(response)

✅ Created YouTube MCP instance
🤖 Agent initialized with model: accounts/fireworks/models/qwen2p5-72b-instruct
🛠️ Calling tool: get_youtube_video_transcript with args: {'url': 'https://www.youtube.com/watch?v=kPXvf2-C_Hs'}
### Summary of the Video: "How This AI Startup Grew by 100x in Just 6 Months | Fireworks AI, Lin Qiao"

**Title:** How This AI Startup Grew by 100x in Just 6 Months | Fireworks AI, Lin Qiao  
**Published:** August 20, 2024  
**Channel:** EO  
**Duration:** 11 minutes 55 seconds  
**Views:** 77,278  
**Likes:** 1,650  
**Comments:** 51

#### **0:00 - 1:00: Introduction**
- Introduction to Fireworks AI, one of the fastest-growing AI startups in the industry.
- Over the past six months, Fireworks AI has seen a 100x increase in traffic.
- In July, the company closed a Series B funding round led by Sequoia, bringing its valuation to $552 million.

#### **1:00 - 3:30: Lin Qiao's Background**
- Lin Qiao, the founder of Fireworks AI, previously served as the Head of PyTorch.


## Use Case 2: Send Email

**Note**: Gmail integration requires OAuth authentication, so you'll need to authorize the application in your browser. 

In [ ]:
import webbrowser

# Create Gmail MCP server instance
gmail_mcp_instance = mcp_handle_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
)

# Redirect to Gmail OAuth page
webbrowser.open(gmail_mcp_instance.oauth_url)
print(f"🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: {gmail_mcp_instance.oauth_url}")


In [30]:
EMAIL_SUBJECT = "Hello, World!"
EMAIL_BODY = "This is a test email."
EMAIL_RECIPIENT = "zihaolin@mcp-handle.dev" # replace with your email

# Create an agent with Gmail MCP server
agent = Agent(fireworks_client, mcp_handle_client, gmail_mcp_instance.server_url)

response = agent.process_request(
    f"Send an email to {EMAIL_RECIPIENT} with subject {EMAIL_SUBJECT} and body {EMAIL_BODY}"
)

print(response)

The email has been sent successfully to zihaolin@mcp-handle.dev with the subject "Hello, World!" and the body "This is a test email." The email ID is 1977620de59daa96.


## Summary

This tutorial demonstrated how to integrate Fireworks AI's function calling capabilities with MCPHandle MCP servers to create powerful AI applications. We covered two practical examples:

**🎥 YouTube Integration**: Built an AI assistant that can automatically summarize YouTube videos by extracting transcripts and providing detailed, timestamped summaries.

**📧 Gmail Integration**: Created an AI-powered email assistant that can send emails through Gmail with OAuth authentication.

### Key Takeaways:
- **Easy Setup**: MCPHandle MCP servers can be created with just a few lines of code using the official SDK
- **Fireworks AI Compatible**: All tools are formatted for seamless Fireworks AI function calling
- **Versatile**: Support for both simple APIs (YouTube) and OAuth-authenticated services (Gmail)
- **Scalable**: The same pattern can be applied to any of the 100+ MCP servers available in MCPHandle

### 🚀 Next Steps
- **Explore More MCP Servers**: Try other available servers like Slack, Notion, CRM, etc.
- **Try Different Fireworks AI Models**: Experiment with various models like Llama, Mixtral, or Deepseek for different use cases
- **Build Complex Multi-Server Workflows**: Create sophisticated agents that combine Gmail + Slack + Notion for complete business automation
- **Production Deployment**: Scale these patterns for production applications

### 🔗 Useful Resources
- [Fireworks AI Documentation](https://docs.fireworks.ai/)
- [MCP Handle Documentation](https://www.mcp-handle.dev/docs/)
- [MCP Protocol Specification](https://modelcontextprotocol.io/)

**Happy building!** 🚀
